In [5]:
from sklearn.preprocessing import OneHotEncoder
from flask import Flask,render_template,request
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from scipy.stats import zscore
from sklearn.ensemble import StackingRegressor
from sklearn.neural_network import MLPRegressor
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime,timedelta
from sklearn.metrics import mean_absolute_error

In [6]:
def posix_time(dt):
    return (dt-datetime(1970,1,1))/timedelta(seconds=1)

In [7]:
import pandas as pd

data=pd.read_csv("traffic_volume_data.csv")
print(data)

                 date_time  is_holiday  air_pollution_index  humidity  \
0      2012-10-02 21:00:00           1                  128        61   
1      2012-10-02 22:00:00           1                   18        61   
2      2012-10-02 23:00:00           1                  186        60   
3      2012-10-03 00:00:00           1                   58        60   
4      2012-10-03 01:00:00           1                  158        59   
...                    ...         ...                  ...       ...   
33727  2017-05-17 21:00:00           1                  269        85   
33728  2017-05-17 22:00:00           1                   81        70   
33729  2017-05-17 22:00:00           1                  138        70   
33730  2017-05-17 23:00:00           1                  229        66   
33731  2017-05-17 23:00:00           1                  140        63   

       wind_speed  wind_direction  visibility_in_miles  dew_point  \
0               4             325                    4

In [29]:
data=data.sort_values(by=["date_time"],ascending=True).reset_index(drop=True)
last_n_hours=[1,2,3,4,5,6]
for n in last_n_hours:
    data[f"last_{n}_hour_traffic"]=data["traffic_volume"].shift(n)
data=data.dropna().reset_index(drop=True)
data.loc[data["is_holiday"]!='None','is_holiday']=1
data.loc[data["is_holiday"]=='None','is_holiday']=0
data["is_holiday"]=data["is_holiday"].astype(int)



data["date_time"]=pd.to_datetime(data["date_time"])
data["hour"]=data["date_time"].map(lambda x: int (x.strftime("%H")))
data["month_day"]=data["date_time"].map(lambda x: int (x.strftime("%d")))
data["weekday"]=data["date_time"].map(lambda x: x.weekday()+1)
data["month"]=data["date_time"].map(lambda x: int (x.strftime("%m")))
data["year"]=data["date_time"].map(lambda x: int (x.strftime("%Y")))
data.to_csv("traffic_volume_data.csv",index=None)
data


,date_time,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,...,last_2_hour_traffic,last_3_hour_traffic,last_4_hour_traffic,last_5_hour_traffic,last_6_hour_traffic,hour,month_day,weekday,month,year
0,2012-10-04 08:00:00,1,90,50,4,360,5,5,281.86,0.0,...,367.0,1015.0,3591.0,5337.0,814.0,8,4,4,10,2012
1,2012-10-04 09:00:00,1,144,75,0,0,6,6,284.98,0.0,...,835.0,367.0,1015.0,3591.0,5337.0,9,4,4,10,2012
2,2012-10-04 18:00:00,1,234,45,3,190,5,5,296.36,0.0,...,5985.0,835.0,367.0,1015.0,3591.0,18,4,4,10,2012
3,2012-10-04 19:00:00,1,220,37,3,170,6,6,293.85,0.0,...,5309.0,5985.0,835.0,367.0,1015.0,19,4,4,10,2012
4,2012-10-04 20:00:00,1,210,28,3,190,5,5,292.43,0.0,...,4907.0,5309.0,5985.0,835.0,367.0,20,4,4,10,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9983,2017-05-17 17:00:00,1,218,59,2,334,6,6,292.09,0.0,...,5590.0,4789.0,4990.0,4517.0,4517.0,17,17,3,5,2017
9984,2017-05-17 18:00:00,1,91,67,2,337,8,8,291.35,0.0,...,6426.0,5590.0,4789.0,4990.0,4517.0,18,17,3,5,2017
9985,2017-05-17 19:00:00,1,27,84,1,321,4,4,290.56,0.0,...,5763.0,6426.0,5590.0,4789.0,4990.0,19,17,3,5,2017
9986,2017-05-17 22:00:00,1,81,70,1,24,2,2,286.95,0.0,...,4864.0,5763.0,6426.0,5590.0,4789.0,22,17,3,5,2017


In [9]:
sns.set()
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False
warnings.filterwarnings("ignore")
data=pd.read_csv("traffic_volume_data.csv")
data=data.sample(9994).reset_index(drop=True)
label_columns=["weather_type","weather_description"]
numeric_columns=["is_holiday","temperature","weekday","hour","month_day","year","month"]


In [10]:
features=numeric_columns+label_columns
x=data[features]
x.head()

,is_holiday,temperature,weekday,hour,month_day,year,month,weather_type,weather_description
0,1,261.79,1,9,8,2016,2,Haze,haze
1,1,276.02,6,5,20,2016,2,Clear,sky is clear
2,1,291.46,6,14,27,2016,8,Drizzle,light intensity drizzle
3,1,258.44,5,8,3,2017,2,Clear,sky is clear
4,1,269.72,4,4,13,2014,3,Clouds,overcast clouds


In [11]:
x.shape

(9994, 9)

In [12]:
data['weather_type'].unique()

array(['Haze', 'Clear', 'Drizzle', 'Clouds', 'Snow', 'Rain', 'Fog',
       'Mist', 'Thunderstorm', 'Smoke', 'Squall'], dtype=object)

In [13]:
data['weather_description'].unique()

array(['haze', 'sky is clear', 'light intensity drizzle',
       'overcast clouds', 'scattered clouds', 'light snow', 'light rain',
       'fog', 'mist', 'broken clouds', 'heavy snow', 'Sky is Clear',
       'thunderstorm', 'drizzle', 'proximity thunderstorm',
       'moderate rain', 'few clouds', 'heavy intensity rain',
       'thunderstorm with heavy rain', 'snow', 'smoke',
       'thunderstorm with light rain', 'proximity thunderstorm with rain',
       'very heavy rain', 'thunderstorm with rain',
       'proximity shower rain', 'light intensity shower rain',
       'heavy intensity drizzle', 'thunderstorm with light drizzle',
       'light rain and snow', 'SQUALLS', 'freezing rain',
       'light shower snow', 'proximity thunderstorm with drizzle'],
      dtype=object)

In [14]:
data.loc[ data["weather_type"]=='Clear','weather_type']=1
data.loc[data["weather_type"]=='Clouds','weather_type']=2
data.loc[ data["weather_type"]=='Snow','weather_type']=3
data.loc[data["weather_type"]=='Haze','weather_type']=4
data.loc[ data["weather_type"]=='Mist','weather_type']=5
data.loc[data["weather_type"]=='Fog','weather_type']=6
data.loc[ data["weather_type"]=='Drizzle','weather_type']=7
data.loc[data["weather_type"]=='Rain','weather_type']=8
data.loc[ data["weather_type"]=='Thunderstorm','weather_type']=9
data.loc[data["weather_type"]=='Squall','weather_type']=10
data.loc[ data["weather_type"]=='Smoke','weather_type']=11

In [15]:
data.loc[ data["weather_description"]=='sky is clear','weather_description']=1
data.loc[ data["weather_description"]=='few clouds','weather_description']=2
data.loc[ data["weather_description"]=='scattered clouds','weather_description']=3
data.loc[ data["weather_description"]=='heavy snow','weather_description']=4
data.loc[ data["weather_description"]=='overcast clouds','weather_description']=5
data.loc[ data["weather_description"]=='broken clouds','weather_description']=6
data.loc[ data["weather_description"]=='haze','weather_description']=7
data.loc[ data["weather_description"]=='light snow','weather_description']=8
data.loc[ data["weather_description"]=='mist','weather_description']=9
data.loc[ data["weather_description"]=='fog','weather_description']=10
data.loc[ data["weather_description"]=='snow','weather_description']=11
data.loc[ data["weather_description"]=='drizzle','weather_description']=12
data.loc[ data["weather_description"]=='light intensity drizzle','weather_description']=13
data.loc[ data["weather_description"]=='moderate rain','weather_description']=14
data.loc[ data["weather_description"]=='light rain','weather_description']=15
data.loc[ data["weather_description"]=='heavy intensity drizzle','weather_description']=16
data.loc[ data["weather_description"]=='Sky is Clear','weather_description']=17
data.loc[ data["weather_description"]=='light rain and snow','weather_description']=18
data.loc[ data["weather_description"]=='proximity shower rain','weather_description']=19
data.loc[ data["weather_description"]=='proximity thunderstorm','weather_description']=20
data.loc[ data["weather_description"]=='heavy intensity rain','weather_description']=21
data.loc[ data["weather_description"]=='thunderstorm','weather_description']=22
data.loc[ data["weather_description"]=='thunderstorm with light rain','weather_description']=23
data.loc[ data["weather_description"]=='proximity thunderstorm with rain','weather_description']=24
data.loc[ data["weather_description"]=='proximity thunderstorm with drizzle','weather_description']=25
data.loc[ data["weather_description"]=='thunderstorm with rain','weather_description']=26
data.loc[ data["weather_description"]=='thunderstorm with heavy rain','weather_description']=27
data.loc[ data["weather_description"]=='thunderstorm with drizzle','weather_description']=28
data.loc[ data["weather_description"]=='freezing rain','weather_description']=29
data.loc[ data["weather_description"]=='shower drizzle','weather_description']=30
data.loc[ data["weather_description"]=='very heavy rain','weather_description']=31
data.loc[ data["weather_description"]=='sleet','weather_description']=32
data.loc[ data["weather_description"]=='light shower snow','weather_description']=33
data.loc[ data["weather_description"]=='shower snow','weather_description']=34
data.loc[ data["weather_description"]=='thunderstorm with light drizzle','weather_description']=35
data.loc[ data["weather_description"]=='light intensity shower rain','weather_description']=36
data.loc[ data["weather_description"]=='SQUALLS','weather_description']=37
data.loc[ data["weather_description"]=='smoke','weather_description']=38



In [16]:
label_columns=["weather_type","weather_description"]
numeric_columns=["is_holiday","temperature","weekday","hour","month_day","year","month"]
features=numeric_columns+label_columns
x=data[features]
x

,is_holiday,temperature,weekday,hour,month_day,year,month,weather_type,weather_description
0,1,261.79,1,9,8,2016,2,4,7
1,1,276.02,6,5,20,2016,2,1,1
2,1,291.46,6,14,27,2016,8,7,13
3,1,258.44,5,8,3,2017,2,1,1
4,1,269.72,4,4,13,2014,3,2,5
...,...,...,...,...,...,...,...,...,...
9989,1,279.00,7,9,18,2012,11,4,7
9990,1,291.00,1,22,29,2015,6,5,9
9991,1,283.88,1,15,3,2017,4,8,15
9992,1,296.50,7,2,24,2016,7,9,23


In [17]:
y=data["traffic_volume"]
y

0       4724
1        641
2       4224
3       5861
4        841
        ... 
9989    2561
9990    1680
9991    5341
9992       5
9993     295
Name: traffic_volume, Length: 9994, dtype: int64

In [18]:
from sklearn.model_selection import train_test_split
trainX,testX,trainY,testY=train_test_split(x,y,test_size=0.2)

from sklearn.neural_network import MLPRegressor
regre=MLPRegressor(random_state=1,max_iter=500).fit(trainX,trainY)
y_pred=regre.predict(testX)
y_pred


array([3702.70288155,  984.22146368, 1743.04866757, ..., 1942.09847952,
       3586.8990208 , 3568.48385956])

In [19]:
testY

843     4688
8667     588
2883     469
6040    2759
5320    2578
        ... 
4754     755
8698     648
7633     618
4210    4302
4818    6115
Name: traffic_volume, Length: 1999, dtype: int64

In [28]:
import numpy as np
ip=np.array([[0,276.02	,6	,5	,20	,2016	,2	,1	,1]])
#ip=().reshape(1,-1)
out=regre.predict(ip)
out

array([1959.65280783])

In [26]:

from sklearn.metrics import mean_absolute_error
#trainX,testX,trainY,testY=train_test_split(X,y,test_size=0.2)
y_pred=regre.predict(testX)
print("mean absolute error = ",mean_absolute_error(y_pred,testY))
print(trainX.shape)

mean absolute error =  1063.661176640941
(7995, 9)
